In [4]:
from NuRadioMC.utilities import medium
from NuRadioMC.SignalProp.radioproparaytracing import radiopropa_ray_tracing
from AntPosCal.time_differences.travel_time_simulator_radiopropa import TravelTimeSimulatorRadioPropa
import mattak.Dataset  # RNO-G data handling
import time
import numpy as np
import argparse
import uproot  # ROOT file reading
import matplotlib.pyplot as plt
import gpxpy  # GPS track parsing
import gpxpy.gpx
import gzip
import pandas as pd
import os
import readRNOGDataMattak 
import logging
import requests
from NuRadioReco.detector import detector
from rnog_analysis_tools.coordinate_system.coordinate_system import CoordinateSystem
from datetime import datetime, timezone
from pathlib import Path
from __future__ import annotations


def load_balloon_track(file_path: str | Path,
                       start_time: datetime | None = None,
                       end_time:   datetime | None = None,
                       *,
                       tzinfo = timezone.utc) -> pd.DataFrame:
    """
    Load a balloon coordinate file with columns:
        Time(UTC)  Latitude  Longitude  Height(m)  ZenithAngle(deg)  X(m)  Y(m)

    If BOTH start_time and end_time are provided (datetimes), return only rows
    within [start_time, end_time] inclusive. Otherwise, return all rows.

    The 'Time(UTC)' column is parsed as datetime and made tz-aware using `tzinfo`
    if it was naïve in the file.
    """
    file_path = Path(file_path).expanduser()

    # ---------- read whole file ----------
    df = pd.read_csv(
        file_path,
        sep=r"\t+",                # split on tabs or multiple spaces
        engine="python",           # allows regex separator
        parse_dates=["Time(UTC)"],
        dtype={
            "Latitude": float,
            "Longitude": float,
            "Height(m)": float,
            "ZenithAngle(deg)": float,
            "X(m)": float,
            "Y(m)": float,
        },
    )

    # ---------- make the time column tz-aware ----------
    if df["Time(UTC)"].dt.tz is None:  # still naïve
        df["Time(UTC)"] = df["Time(UTC)"].dt.tz_localize(tzinfo)

    # ---------- normalise user-supplied bounds ----------
    def _norm(dt: datetime | None):
        if dt is None:
            return None
        return dt if dt.tzinfo is not None else dt.replace(tzinfo=tzinfo)

    start_time = _norm(start_time)
    end_time   = _norm(end_time)

    # ---------- optional filtering (only if BOTH bounds are given) ----------
    if start_time is not None and end_time is not None:
        mask = (df["Time(UTC)"] >= start_time) & (df["Time(UTC)"] <= end_time)
        df = df.loc[mask].copy()

    # Return as-is (keep 'Time(UTC)' as a column)
    # Columns: Time(UTC), Latitude, Longitude, Height(m), ZenithAngle(deg), X(m), Y(m)
    return df



t0 = datetime(2022, 9, 29, 23, 35, tzinfo=timezone.utc)
t1 = datetime(2022, 9, 29, 23, 38, tzinfo=timezone.utc)

balloon_dis = load_balloon_track('../balloon_minute_coords_zenith_DF.txt', t0, t1)

balloon_dis_every_30s = (
    balloon_dis
    .sort_values('Time(UTC)')
    .iloc[::10]              # rows 0, 10, 20, ...
    .reset_index(drop=True)
)
print(balloon_dis_every_30s)

                  Time(UTC)   Latitude  Longitude  Height(m)  \
0 2022-09-29 23:35:00+00:00  72.652036 -38.600241    4444.20   
1 2022-09-29 23:35:30+00:00  72.653766 -38.603899    4555.37   
2 2022-09-29 23:36:00+00:00  72.655615 -38.607768    4672.64   
3 2022-09-29 23:36:30+00:00  72.657347 -38.610726    4784.97   
4 2022-09-29 23:37:00+00:00  72.659161 -38.613529    4896.09   
5 2022-09-29 23:37:30+00:00  72.660934 -38.616786    5009.46   
6 2022-09-29 23:38:00+00:00  72.662707 -38.620846    5120.57   

   ZenithAngle(deg)      X(m)      Y(m)  
0             62.27 -4814.344  7742.189  
1             62.32 -4935.890  7935.913  
2             62.37 -5064.420  8142.973  
3             62.38 -5162.598  8336.891  
4             62.40 -5255.570  8539.967  
5             62.42 -5363.668  8738.518  
6             62.48 -5498.500  8937.151  


In [5]:


t0 = time.time()

det = detector.Detector(source="rnog_mongo")
det.update(datetime(2024, 6, 1))
#station_deployed=[11,12,13,21,22,23,24]
station=21
station_dis=det.get_absolute_position(station)
ch_arr=[]
coordsys=CoordinateSystem()

# # Instead of using pulser location from detector,
# # use user-defined coordinate (Lat, Lon, Height)
# user_x = -417.927690   # meters
# user_y = 458.540055   # meters
# user_height = 337.32  # meters

balloon_dis=[]
for index in range(balloon_dis_every_30s.shape[0]):
    balloon_x=balloon_dis_every_30s.at[index,"X(m)"]
    balloon_y=balloon_dis_every_30s.at[index,"Y(m)"]
    balloon_z=balloon_dis_every_30s.at[index,"Height(m)"]
    balloon_dis.append([balloon_x, balloon_y, balloon_z])

print(f"Using user-defined coordinate: ENU = {balloon_dis}")
print(balloon_dis[0])

for ch in range(24):
    ch_dis=station_dis + det.get_relative_position(station,ch)
    #print(ch_dis)
    ch_arr.append(np.array(ch_dis))
    #coord=coordsys.enu_to_geodetic(ch_dis[0],ch_dis[1],ch_dis[2])
    #ch_coord.append(np.array(coord))
print(ch_arr)


det1 = detector.Detector(json_filename="/users/PAS2608/youwei/RNO_G/NuRadioMC/NuRadioReco/detector/RNO_G/RNO_season_2024.json")
det1.update(datetime(2024, 9, 14))

ch_index=np.array(range(24))


#-------------------------------------------------------------------------------------------------------------------
ice_simp = medium.greenland_simple()
ice_poly = medium.greenland_poly5()
ice_simp_propa=TravelTimeSimulatorRadioPropa(ice_simp)
ice_poly_propa=TravelTimeSimulatorRadioPropa(ice_poly)
#ice_poly_propa._TravelTimeSimulatorRadioPropa__raytracer.set_maximum_trajectory_length(20000)

print("data successfully loaded in")
#-------------------------------------------------------------------------------------------------------------------
# Using NumPy zeros
matrix_poly = pd.DataFrame(np.zeros((24, 24)))

sphere_sizes_i = np.array([250.,25., 2., .5])
#sphere_sizes_i = np.array([25., 2., .5,.1])
ice_poly_propa.set_sphere_sizes(sphere_sizes_i)
ice_simp_propa.set_sphere_sizes(sphere_sizes_i)
print(ch_arr[6])
print(balloon_dis[0])

# for i in ch_index:
#     for j in ch_index:
#         if i > j:
#             try:


#                 # Calculate travel time for channel i

#                 #ice_poly_propa.set_sphere_sizes(sphere_sizes_i)
#                 time_i = ice_poly_propa.get_travel_time( balloon_dis[0], ch_arr[i])
                
#                 # Calculate travel time for channel j
#                 #ice_poly_propa.set_sphere_sizes(sphere_sizes_j)
#                 time_j = ice_poly_propa.get_travel_time( balloon_dis[0],ch_arr[j])
                
#                 # Compute time difference
#                 time_diff = time_i - time_j
#                 matrix_poly.iloc[i, j] = time_diff

#                 # if i ==7:
#                 #     print(ch_arr[i])
#                 #     print(time_i)
#                 #     print(time_j)
#                 #     print(time_diff)
                
#                 # Optional: Print for debugging
#                 print(f"Channels ({i},{j}): Time difference = {time_diff}")
#                 #print(f"  Used sphere sizes: {sphere_sizes_i} (ch{i}), {sphere_sizes_j} (ch{j})")
                
#             except Exception as e:
#                 print(f"Error for ({i},{j}): {e}")
#                 matrix_poly.iloc[i, j] = np.nan


# print(matrix_poly)
# matrix_simp = pd.DataFrame(np.zeros((24, 24)))
# print("matrix_poly successfully calculated")


# for i in ch_index:
#     for j in ch_index:
#         if i > j:
#             try:
                
#                 # Calculate travel time for channel i
#                 # ice_simp_propa.set_sphere_sizes(sphere_sizes_i)
#                 time_i = ice_simp_propa.get_travel_time(ch_arr[i], balloon_dis[0])
                
#                 # Calculate travel time for channel j
#                 #ice_simp_propa.set_sphere_sizes(sphere_sizes_j)
#                 time_j = ice_simp_propa.get_travel_time(ch_arr[j], balloon_dis[0])
                
#                 # Compute time difference
#                 time_diff = time_i - time_j
#                 matrix_simp.iloc[i, j] = time_diff
                
#                 # Optional: Print for debugging
#                 print(f"Channels ({i},{j}): Time difference = {time_diff}")
#                 #print(f"  Used sphere sizes: {sphere_sizes_i} (ch{i}), {sphere_sizes_j} (ch{j})")
                
#             except Exception as e:
#                 print(f"Error for ({i},{j}): {e}")
#                 matrix_poly.iloc[i, j] = np.nan
# print("matrix_simp successfully calculated")

                



# #-------------------------------------------------------------------------------------------------------------------
# # Write matrices to file (with station number in filename)
# with open(f'Ice_model/matrix_ice_model_poly_station_{station}_run_2205_zenith66.txt', 'w') as f:
#     f.write("Time delay matrix:\n")
#     f.write(matrix_poly.to_string())

# with open(f'Ice_model/matrix_ice_model_simp_station_{station}_run_2205_zenith66.txt', 'w') as f:
#     f.write("Time delay matrix:\n")
#     f.write(matrix_simp.to_string())


# t_total = time.time() - t0
# print(t_total)

Using user-defined coordinate: ENU = [[np.float64(-4814.344), np.float64(7742.189), np.float64(4444.2)], [np.float64(-4935.89), np.float64(7935.913), np.float64(4555.37)], [np.float64(-5064.42), np.float64(8142.973), np.float64(4672.64)], [np.float64(-5162.598), np.float64(8336.891), np.float64(4784.97)], [np.float64(-5255.57), np.float64(8539.967), np.float64(4896.09)], [np.float64(-5363.668), np.float64(8738.518), np.float64(5009.46)], [np.float64(-5498.5), np.float64(8937.151), np.float64(5120.57)]]
[np.float64(-4814.344), np.float64(7742.189), np.float64(4444.2)]
[array([-309.07682258,  496.701256  ,  -97.59036387]), array([-309.07682258,  496.701256  ,  -96.59036387]), array([-309.07682258,  496.701256  ,  -95.58036387]), array([-309.07682258,  496.701256  ,  -94.58036387]), array([-309.07682258,  496.701256  ,  -92.61036387]), array([-309.07682258,  496.701256  ,  -80.64036387]), array([-309.07682258,  496.701256  ,  -60.20036387]), array([-309.07682258,  496.701256  ,  -40.55036

In [6]:
def compute_time_matrix_for_source(propa, src_point, ch_arr, ch_index):
    """
    Compute lower-triangular time-difference matrix for one source point.
    We compute each channel's travel time once, then reuse to fill i>j.
    """
    N = len(ch_arr)
    mat = pd.DataFrame(np.zeros((N, N)))

    # 1) Compute travel time to 'src_point' from every channel once
    times = np.empty(N, dtype=float)
    for i in ch_index:
        try:
            times[i] = propa.get_travel_time(src_point,ch_arr[i]) + propa.get_travel_time(ch_arr[i],src_point)
        except Exception as e:
            print(f"Error computing time for ch {i} @ src={src_point}: {e}")
            times[i] = np.nan

    # 2) Fill lower triangle: dt(i,j) = t_i - t_j for i>j
    for i in ch_index:
        for j in ch_index:
            if i > j:
                ti, tj = times[i], times[j]
                mat.iloc[i, j] = ti - tj if np.all(np.isfinite([ti, tj])) else np.nan

    return mat

# ---------------------------------------------------------------
# Loop over every balloon position and write two files each time
# ---------------------------------------------------------------
for k, src in enumerate(balloon_dis):
    # src is [x, y, z] for this time step
    try:
        matrix_poly = compute_time_matrix_for_source(ice_poly_propa, src, ch_arr, ch_index)
        matrix_simp = compute_time_matrix_for_source(ice_simp_propa, src, ch_arr, ch_index)

        # Write matrices to file (index in filename)
        out_poly = f'Ice_model/matrix_ice_model_poly_station_{station}_run_2205_idx{k:01d}_sum.txt'
        out_simp = f'Ice_model/matrix_ice_model_simp_station_{station}_run_2205_idx{k:01d}_sum.txt'

        with open(out_poly, 'w') as f:
            f.write("Time delay matrix:\n")
            f.write(matrix_poly.to_string())

        with open(out_simp, 'w') as f:
            f.write("Time delay matrix:\n")
            f.write(matrix_simp.to_string())

        print(f"[{k+1}/{len(balloon_dis)}] wrote files:\n  {out_poly}\n  {out_simp}")

    except Exception as e:
        print(f"Top-level error at balloon idx {k}, src={src}: {e}")

t_total = time.time() - t0
print(f"Total elapsed: {t_total:.2f} s")

[1/7] wrote files:
  Ice_model/matrix_ice_model_poly_station_21_run_2205_idx0_sum.txt
  Ice_model/matrix_ice_model_simp_station_21_run_2205_idx0_sum.txt
[2/7] wrote files:
  Ice_model/matrix_ice_model_poly_station_21_run_2205_idx1_sum.txt
  Ice_model/matrix_ice_model_simp_station_21_run_2205_idx1_sum.txt
[3/7] wrote files:
  Ice_model/matrix_ice_model_poly_station_21_run_2205_idx2_sum.txt
  Ice_model/matrix_ice_model_simp_station_21_run_2205_idx2_sum.txt
[4/7] wrote files:
  Ice_model/matrix_ice_model_poly_station_21_run_2205_idx3_sum.txt
  Ice_model/matrix_ice_model_simp_station_21_run_2205_idx3_sum.txt
[5/7] wrote files:
  Ice_model/matrix_ice_model_poly_station_21_run_2205_idx4_sum.txt
  Ice_model/matrix_ice_model_simp_station_21_run_2205_idx4_sum.txt
[6/7] wrote files:
  Ice_model/matrix_ice_model_poly_station_21_run_2205_idx5_sum.txt
  Ice_model/matrix_ice_model_simp_station_21_run_2205_idx5_sum.txt
[7/7] wrote files:
  Ice_model/matrix_ice_model_poly_station_21_run_2205_idx6_sum.